# Cleaning - Simply Recipes Ingredients List

Sources: 
* https://htmlpreview.github.io/?https://github.com/kulsoom-abdullah/kulsoom-abdullah.github.io/blob/master/AWS-lambda-implementation/model_implementation/recipe%20binary%20classification/recipe%20binary%20classification.html#Words-to-Remove
* https://towardsdatascience.com/https-towardsdatascience-com-end-to-end-recipe-cuisine-classification-e97f4ac22104


NYT - Natural Language Processor for Ingredients 

* https://github.com/nytimes/ingredient-phrase-tagger
* https://homepages.inf.ed.ac.uk/csutton/publications/crftutv2.pdf

CRF Tutorial 
* https://www.analyticsvidhya.com/blog/2018/08/nlp-guide-conditional-random-fields-text-classification/
* http://acepor.github.io/2017/03/06/CRF-Python/
* http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/
* https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html

In [1]:
import pandas as pd
import numpy as np
import re
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/markishab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
recipes_df = pd.read_pickle('../../data/02_intermediate/recipies_sr_inter.pkl')

Let's take a look at our two datasets

In [3]:
recipes_df.reset_index(inplace=True, drop=True)

In [6]:
recipes_df

,title,prep_time,cook_time,recipe_yield,tags,ingredients,entire_card,recipe_links
0,Grilled Cheese BLT,10 minutes,10 minutes,4 sandwiches,"'Dinner', 'Lunch', 'Sandwich', 'Favorite Summe...","[8 slices sourdough bread, 4 tablespoon unsalt...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/grilled_...
1,Pulled Pork Sandwich,10 minutes,"2 hours, 45 minutes",Serves 6 to 8,"'Dinner', 'Sandwich', 'Budget', 'Comfort Food'...","[ , 1 large onion, chopped, 6 garlic cloves, p...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/pulled_p...
2,How to Make Bacon in the Oven,5 minutes,20 minutes,12 strips,"'Tips', 'Breakfast and Brunch', 'Baking', 'How...","[12 strips bacon, 1/2 teaspoon ground black pe...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/how_to_m...
3,Sausage Stuffed Zucchini,15 minutes,1 hour,Serves 4,"'Dinner', 'Favorite Summer', 'Make-ahead', 'It...","[2 tablespoons extra virgin olive oil, 1/2 pou...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/italian_...
4,The Best Dry Rub for Ribs,5 minutes,NaN,NaN,"'Favorite Fall', 'Favorite Summer', 'Game Day'...",[3/4 cup packed dark brown sugar (or 1/2 cup i...,"['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/the_best...
5,Ginger Pork Rice Bowls,15 minutes,20 minutes,4-6 servings,"'Eat Your Food', 'Dinner', 'Kid-friendly', 'Ma...","[ , 1 tablespoon olive oil, 1/2 cup diced whit...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/ginger_p...
6,Hawaiian SPAM Tacos with Pineapple,10 minutes,30 minutes,"12 small tacos, serving 4 adults","'Eat Your Food', 'Dinner', 'Kid-friendly', 'Ha...","[ , 1 habañero pepper, seeded and minced, 1/4 ...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/hawaiian...
7,Air Fryer Chinese Egg Rolls,20 minutes,25 minutes,12 egg rolls,"'Appetizer', 'Snack', 'Air Fryer', 'Quick and ...","[ , 1 tablespoon olive oil, 1 pound ground por...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/air_frye...
8,Grilled Bacon-Wrapped Stuffed Hot Dogs,10 minutes,11 minutes,Makes 4 stuffed hot dogs,"'Dinner', 'Favorite Summer', 'Grill', 'Bacon',...","[1 teaspoon ketchup, 1 teaspoon Dijon mustard,...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/grilled_...
9,Italian Grilled Cheese Sandwiches,10 minutes,20 minutes,4 sandwiches,"'Eat Your Food', 'Dinner', 'Sandwich', 'Kid-fr...","[ , 1 cup hot pickled cauliflower, chopped, 1/...","['\n\n ', '\n ...",https://www.simplyrecipes.com/recipes/italian_...


## let's clean the list so that it can be used in a crf model 

In [7]:
ingredients_lists = list(recipes_df.ingredients)

In [23]:
len(ingredients_lists)

1763

In [24]:
ingredients_lists

[['8 slices sourdough bread',
  '4 tablespoon unsalted butter, at room temperature',
  '8 ounces (2 cups) shredded cheddar cheese',
  '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
  "8 to 12 slices ', 'cooked bacon",
  '12 leaves butterhead or other crispy lettuce',
  'Kosher salt and black pepper',
  ' '],
 [' ',
  '1 large onion, chopped',
  '6 garlic cloves, peeled',
  '1 pickled jalapeño pepper, seeded and chopped',
  '2 teaspoons Chipotle chile powder',
  '1 tablespoon tomato paste',
  '2 tablespoon Dijon mustard',
  '3/4 cup distilled white vinegar',
  '1 teaspoon paprika',
  '1/3 cup ketchup',
  '2 teaspoons Worcestershire sauce',
  '1/4 cup light brown sugar',
  'Salt',
  ' ',
  'For the pork:',
  ' ',
  '1 bay leaf',
  '3 pounds pork butt shoulder roast',
  'Hamburger buns',
  ' '],
 ['12 strips bacon', '1/2 teaspoon ground black pepper (optional)', ' '],
 ['2 tablespoons extra virgin olive oil',
  '1/2 pound Italian sausage, 

Let's isolate the actual ingredients

In [41]:
for element in ingredients_lists:
    try:
        while(" " in element) : 
            element.remove(" ")
    except:
        pass

Replace ```'\\\\xa0'``` with space

In [36]:
new_list = []
for ingr_list in ingredients_lists:
    ingr_list = [word.replace('\\\\xa0', ' ') for word in ingr_list]
    new_list.append(ingr_list)

In [37]:
len(new_list)

1763

In [38]:
final_list = []
for il in new_list:
    il = [word.replace('\\xa0', ' ') for word in il]
    final_list.append(il)

In [39]:
len(final_list)

1763

In [54]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            recipe.pop(idx)

In [56]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            print(ingredient)

In [57]:
len(final_list)

1763

In [63]:
recipes_df.ingredients[70]

['1 tablespoons extra virgin olive oil',
 '1 yellow onion,\\\\xa0finely chopped (about 1\\\\xa01/2 to 2 cups)',
 '2 cloves garlic, minced (about 2 teaspoons)',
 '4 ounces (110 g) Italian sweet sausage (or one 7 to 8 inch link)',
 '4 ounces (110 g) \\\\xa0Italian hot and spicy sausage\\\\xa0(or one 7 to 8 inch link)',
 '1\\\\xa0large (28 oz, 794 g) can of whole peeled tomatoes',
 '1 pound (16 oz, 450 g) spaghetti pasta',
 'Salt',
 'Freshly grated Parmesan cheese']

In [64]:
final_list[70]

['1 tablespoons extra virgin olive oil',
 '1 yellow onion, finely chopped (about 1 1/2 to 2 cups)',
 '2 cloves garlic, minced (about 2 teaspoons)',
 '4 ounces (110 g) Italian sweet sausage (or one 7 to 8 inch link)',
 '4 ounces (110 g)  Italian hot and spicy sausage (or one 7 to 8 inch link)',
 '1 large (28 oz, 794 g) can of whole peeled tomatoes',
 '1 pound (16 oz, 450 g) spaghetti pasta',
 'Salt',
 'Freshly grated Parmesan cheese']

Make titles into list 

In [44]:
# food_titles = list(recipes_df.title)

In [45]:
# recipe_dict = dict()
# for title, recipes in zip(food_titles, final_list):
# #     print(title)
#     recipe_dict[title] = recipes

In [69]:
len(recipes_df)

1763

In [70]:
recipes_df['ingredient'] = final_list

In [78]:
recipes_df.to_pickle('../../data/02_intermediate/recipes_sr_final.pickle')